# PREPARATIONS NEEDED TO RUN JOAKIM'S CODE
- assumes that "HeatStressCleanWorkFlow.ipynb" has been run to generate "MY_weather.csv", "MY_weather_filtered.csv" and "updateDF.csv"
- also create "../Data/MergedData/" folder and place data here

In [20]:
from datetime import timedelta
import pandas as pd
import numpy as np

In [21]:
#
#
# Make YS
# df = pd.read_csv('../Data/MergedData/MY_weather.csv', low_memory=False)
df = pd.read_csv('../Data/MergedData/MY_weather_filtered.csv', low_memory=False)
print(df.shape)
df['StartDate'] = pd.to_datetime(df['StartDate'])


# Function to determine the season based on month
def get_season(date):
    month = date.month
    if month in [12, 1, 2]:
        return 1
    elif month in [3, 4, 5]:
        return 2
    elif month in [6, 7, 8]:
        return 3
    elif month in [9, 10, 11]:
        return 4


# Apply the function to create a 'YearSeason' variable
df['YearSeason'] = df['StartDate'].apply(lambda x: f"{x.year}0{get_season(x)}")

count_THI = df['THI_adj'].count()
print(count_THI)
count_THI = df['MeanTHI_adj'].count()  # Use MeanTHI_adj!
print(count_THI)

df["THI_adj2"] = df['THI_adj'].fillna(df['MeanTHI_adj'])
df["Temperature2"] = df['Temperature'].fillna(df['MeanTemperature'])

# reorder df - for filtered
col_keep = ["FarmName_Pseudo", 'SE_Number', 'AnimalNumber', 'Del_Cow_Id', 'Breed', 'CalvingDate', 'LactationNumber',
            'DaysInMilk2', 'StartDate', 'StartTime', 'SessionNumber', 'TotalYield', 'YearSeason',
            'Temperature2', 'THI_adj2',
            'RelativeHumidity', 'HW', 'cum_HW', 'Temp15Threshold']
df = df[col_keep]



# reorder df - for unfiltered
# col_keep = ["FarmName_Pseudo", 'SE_Number', 'AnimalNumber', 'Del_Cow_Id', 'Breed', 'CalvingDate', 'LactationNumber',
#            'DaysInMilk', 'StartDate', 'StartTime', 'SessionNumber', 'TotalYield', 'YearSeason',
#            'Temperature', 'RelativeHumidity', 'THI_adj', 'HW', 'cum_HW', 'Temp15Threshold']
# df = df[col_keep]

(895106, 62)
508296
894857


In [22]:
# ADD PARENTAGE
df2 = pd.read_csv("../Data/MergedData/updateDF.csv", low_memory=False)
col_keep = ["SE_Number", "BirthDate", "Father_SE_Number", "Mother_SE_Number", "CullingDate"]
df2 = df2[col_keep]
df2 = df2.drop_duplicates(subset=["SE_Number"])

df3 = pd.merge(df, df2, on="SE_Number", how="left")

In [23]:
# MAKE AGE
df3['BirthDate'] = pd.to_datetime(df3['BirthDate'])
df3['StartDate'] = pd.to_datetime(df3['StartDate'])
df3['Age'] = (df3['StartDate'] - df3['BirthDate']).dt.days

# Filtered
df3.rename(columns={"CullingDate": "CullDecisionDate", "Father_SE_Number": "Father", "Mother_SE_Number": "Mother", "Breed": "BreedName", 
                    "DaysInMilk2": "DaysInMilk"},  inplace=True)

# Unfiltered
# df3.rename(columns={"CullingDate": "CullDecisionDate", "Father_SE_Number": "Father", "Mother_SE_Number": "Mother", "Breed": "BreedName"},  inplace=True)



In [24]:
# Make combined DateTime column
# Ensure StartDate is in datetime format
df3['StartDate'] = pd.to_datetime(df3['StartDate'], errors='coerce')

# Convert StartTime to a timedelta
df3["StartTime2"] = df3["StartTime"]
df3['StartTime2'] = pd.to_timedelta(df3['StartTime2'].astype(str))

# Combine StartDate and StartTime
df3['DateTime'] = df3['StartDate'] + df3['StartTime2']

In [25]:
new_order = ["FarmName_Pseudo", "SE_Number", "AnimalNumber", "StartDate", "StartTime", "LactationNumber", "DaysInMilk", "YearSeason",
             "TotalYield", "DateTime", "BreedName", "BirthDate", "Mother", "Father", "CullDecisionDate", "Temperature2", "RelativeHumidity",
             "THI_adj2", "HW", "cum_HW", "Temp15Threshold", "Age"]
df3 = df3[new_order]

# Filtered
df3.to_csv("../Data/MergedData/CleanedYieldData.csv", index=False)
df3.to_csv("C:/Users/pagd0001/Desktop/Gigacow/Data/20241007/HeatStressEvaluationSummer24-1/Data/MergedData/CleanedYieldData.csv", index=False)

# Unfiltered
# df3.to_csv("../Data/MergedData/YieldData.csv", index=False)
# df3.to_csv("C:/Users/pagd0001/Desktop/Gigacow/Data/20241007/HeatStressEvaluationSummer24-1/Data/MergedData/YieldData.csv", index=False)

In [13]:
# data = pd.read_csv('../Data/MergedData/YieldData.csv', low_memory=False)
df3 = pd.read_csv("../Data/MergedData/CleanedYieldData.csv", low_memory=False)
new_order = ['FarmName_Pseudo', 'SE_Number', 'AnimalNumber', 'StartDate', 'StartTime', 'DateTime', 'LactationNumber', 'DaysInMilk', 'YearSeason', 
             'TotalYield', 'DateTime', 'BreedName', 'Age', 'Mother', 'Father', 'CullDecisionDate', 'Temperature', 'RelativeHumidity', 'THI_adj',
             'HW', 'cum_HW', 'Temp15Threshold']
df3 = df3[new_order]
# df3.to_csv("C:/Users/pagd0001/Desktop/Gigacow/Data/20241007/HeatStressEvaluationSummer24-1/Data/MergedData/YieldData.csv", index=False)
df3.to_csv("C:/Users/pagd0001/Desktop/Gigacow/Data/20241007/HeatStressEvaluationSummer24-1/Data/MergedData/CleanedYieldData.csv", index=False)

In [6]:
# Unfiltered
missing_count = df3["THI_adj"].isna().sum()
print(f"Number of missing values in 'THI_adj': {missing_count}")
print(df3.shape)

Number of missing values in 'THI_adj': 896978
(1669008, 22)


In [19]:
# Filtered
missing_count = df3["THI_adj"].isna().sum()
print(f"Number of missing values in 'THI_adj': {missing_count}")
missing_count = df3["MeanTHI_adj"].isna().sum()
print(f"Number of missing values in 'MeanTHI_adj': {missing_count}")
print(df3.shape)

Number of missing values in 'THI_adj': 386810
Number of missing values in 'MeanTHI_adj': 249
(895106, 24)
